In [1]:
import csv

# Functions

## read_input_poind_config

#### Outline

- `\Zone.csv`
  - $1$行目はヘッダ：地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日

  - $2～9$行目：$1～8$地域の「地域区分, 都市, 緯度, 経度, 日射量ファイル名, 暖房開始日, 暖房終了日, 冷房開始日, 冷房終了日」
    - 暖冷房開始日終了日の書式：$5$桁もしくは$6$桁の数字
      - 気象データの日時表記と同じ
      - 後ろから$2$桁：時刻
      - 後ろから$3～4$桁：日
      - 後ろから$5～6$桁：月
      - 気象データの「暖房$1$冷房$2$」の設定は上書き処理される → `\Zone.csv` の設定が優先

地点データを \Zone.csv から読み込む  
\地域区分+日射量データ窓面入射角特性.xlsx "地域区分"シート
→ \SCFConfig01 下の地点データファイル \Zone.csv を作成 → 読み込み

#### Function

In [2]:
def input_Point(tag):
    
    def convert_tag_to_city(tag):
        if type(tag) == int: # name に数字が指定された場合は省エネ基準の地域区分番号とみなして該当する都市に置き換える
            return {
                '1' : '北見',
                '2' : '岩見沢',
                '3' : '盛岡',
                '4' : '長野',
                '5' : '宇都宮',
                '6' : '岡山',
                '7' : '宮崎',
                '8' : '那覇'
            }[str(tag)]
        else: # その他の場合は都市名が入力されているものとし、変換は行わない
            return tag
    
    ZONE_FILE = "./SCFConfig01/Zone.csv"
    
    city_name = convert_tag_to_city(tag)

    with open(ZONE_FILE, 'r') as f:
        
        reader = csv.reader(f)
        header = next(reader) # Skip header.

        matched_rows = [ row for row in reader if row[1] == city_name ]  # Extract row matched.
        
        if len(matched_rows) == 0:  # In the case that there is now matched data.
            raise IndexError("地点データがありません")
        elif len(matched_rows) > 1:  # In the case that there are more than 1 matched data.
            raise IndexError("該当する地点データが複数見つかりました")
        else:
            matched_row = matched_rows[0]
            return [
                int(matched_row[0]), # region
                matched_row[1],      # city name
                float(matched_row[2]), # latitude
                float(matched_row[3]), # longitude
                matched_row[4],      # file name
                int(matched_row[5]), # start of heating period
                int(matched_row[6]), # end of heating period
                int(matched_row[7]), # start of cooling period
                int(matched_row[8])  # end of cooling period
            ]
            # ここで返るSRFileNameはファイル名のみ

#### Example 1

省エネルギー基準における地域区分の番号で指定した場合

In [3]:
for i in range(1,9):
    print( input_Point( i ) )

[1, '北見', 43.82, 143.91, 'SRforSCF_01.csv', 110100, 43024, 50100, 103124]
[2, '岩見沢', 43.21, 141.78833, 'SRforSCF_02.csv', 110100, 43024, 50100, 103124]
[3, '盛岡', 39.695, 141.168333333333, 'SRforSCF_03.csv', 120100, 33124, 40100, 113024]
[4, '長野', 36.66, 138.195, 'SRforSCF_04.csv', 120100, 33124, 40100, 113024]
[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[6, '岡山', 34.6583333333333, 133.918333333333, 'SRforSCF_06.csv', 120100, 33124, 40100, 113024]
[7, '宮崎', 31.935, 131.416666666667, 'SRforSCF_07.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


#### Example 2

都市名で指定した場合

In [4]:
print(input_Point( "宇都宮"))
print(input_Point( "那覇" ))

[5, '宇都宮', 36.5466666666667, 139.871666666667, 'SRforSCF_05.csv', 120100, 33124, 40100, 113024]
[8, '那覇', 26.2033333333333, 127.688333333333, 'SRforSCF_08.csv', 10100, 33124, 40100, 123124]


#### Example 3

都市名で指定したが該当する名前がリストになかった場合

In [5]:
try:
    input_Point( "東京" ) #データなし
except IndexError as e:
    print(e)

地点データがありません


## input_SRData

#### Outline

- `\SRforSCF_\*\*.csv`
  - \*\*は、$1～8$地域に対応して$01$～$08$が入る(基本)。
    - `\Zone.csv` に追加して対応することは可能。   
    
    
  - 法線面直達日射量、水平面天空日射量、暖房期or冷房期の判別タグ(暖房期:$1$, 冷房期:$2$, 非空調期:$0$)
    - 日射量の単位は$[kcal/(m2h)]$ → 効果係数算定には問題ないのでそのまま使用している
    
    
  - $1$行目はヘッダ：`\SRforSCF_\*\*.csv`(ファイル名), 法線面直達日射量, 水平面天空日射量, 暖房$1$_冷房$2$
  - $2～8762$行目：日時, 法線面直達日射量, 水平面天空日射量, 暖房$1$_冷房$2$
    - $2$行目が$1$月$1$日$0$時、$8762$行目が$12$月$31$日$24$時。$1$時間間隔。全$8761$データ
    - 気象データファイル中の「暖房$1$冷房$2$」の設定は、`\Zone.csv` の設定で上書きされる → 現時点で意味なし
    - $1$列目の日時から「月」を算出

気象データを \SRforSCF_**.csv から読み込む  

\地域区分+日射量データ窓面入射角特性.xlsx "SRforSCF_**.csv"シート 

→ \SCFConfig01 下の地点データファイル \SRforSCF_**.csv を作成 → 読み込み

#### Function

In [6]:
def input_SRData(FileName00, HStart, HEnd, CStart, CEnd):

    # FileName00 = "SRforSCF_**.csv"
    
    Path00 = "./SCFConfig01/"
    
    file_path = Path00 + FileName00
    
    with open(file_path, 'r') as f:
        
        reader = csv.reader(f)
        header = next(reader) # Skip header.
    
        """ \Zone.csv の設定で、暖房期,冷房期,非空調期を割り当て """

        # 元ファイルの4列目はなかったことになる。

        [HeatingPeriod, CoolingPeriod, NonACPeriod] = [1, 2, 0]
        
        rows = []

        for row in reader:
            newrow0 = int(row[0])
            newrow1 = int(row[1])
            newrow2 = int(row[2])
            if CStart <= int(row[0]) <= CEnd:
                newrow3 = CoolingPeriod
            elif CStart > CEnd and CStart <= int(row[0]):
                newrow3 = CoolingPeriod
            elif CStart > CEnd and int(row[0]) <= CEnd:
                newrow3 = CoolingPeriod
            elif HStart <= int(row[0]) <= HEnd:
                newrow3 = HeatingPeriod        
            elif HStart > HEnd and HStart <= int(row[0]):
                newrow3 = HeatingPeriod
            elif HStart > HEnd and int(row[0]) <= HEnd:
                newrow3 = HeatingPeriod
            else:
                newrow3 = NonACPeriod
            rows.append([newrow0, newrow1, newrow2, newrow3])
    
        return rows
            # csv_input は SRHour に渡される

    
#""" 「月」の計算 """
#def calc_Month(MMDDTT):
    
#    Month = MMDDTT // 10000
    
#    return Month

#### Example

In [12]:
for i in range(1,9):
    
    Zone, City, Latitude, Longitude, SRFileName, HStart, HEnd, CStart, CEnd = input_Point(i)
    
    print(SRFileName)
    
    print('Expected start of heating period: ' + str(HStart) )
    print('Expected end of heating period: ' + str(HEnd) )
    print('Expected start of cooling period: ' + str(CStart) )
    print('Expected end of cooling period: ' + str(CEnd) )
    
    SRHour = input_SRData(SRFileName, HStart, HEnd, CStart, CEnd)

    for i in range(1,len(SRHour)-1):
        if SRHour[i][3] != SRHour[i-1][3]:
            print(SRHour[i-1][0], SRHour[i-1][3], SRHour[i][0], SRHour[i][3])    

SRforSCF_01.csv
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
SRforSCF_02.csv
Expected start of heating period: 110100
Expected end of heating period: 43024
Expected start of cooling period: 50100
Expected end of cooling period: 103124
43024 1 50101 2
103124 2 110101 1
SRforSCF_03.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
SRforSCF_04.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 113024
33124 1 40101 2
113024 2 120101 1
SRforSCF_05.csv
Expected start of heating period: 120100
Expected end of heating period: 33124
Expected start of cooling period: 40100
Expected end of cooling period: 1